In [1]:
import warnings
warnings.filterwarnings('ignore')

import api_keys


### 정보 수집 노드
### 웹 검색 노드
### DB 검색 노드
### DB 검색 결과 평가 노드
### 필요 정보 검색 쿼리 작성 노드
### 생성 노드
### 답변 평가 노드
### 쿼리 재작성 노드

In [2]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field

In [11]:
class Info(BaseModel):
    name: str
    universe: str

llm  = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([Info])

In [38]:
from langchain_core.messages import SystemMessage


info_system_prompt = '''Your role is to become a character who engages in conversation with the user.
To do this, you should collect the following information from the user:

- What the character's name is
- What universe(세계관, 영화, 게임 등) does the character belong to

If you cannot determine this information, ask the user directly to clarify. Do not make assumptions.
Once you have all the necessary information, confirm it with the user one more time, and then call the relevant tool.'''

def get_info_messages(messages):
    return [SystemMessage(content=info_system_prompt)] + messages

def info_node(state):
    messages = get_info_messages(state["messages"])
    response = llm_with_tools.invoke(messages)
    return {"messages": response}


In [39]:
state = info_node({"messages": [("user", "쿠엔틴 타란티노 감독의 영화 '바스터즈: 거친 녀석들'에 나온 한스 란다 대령과 대화하고 싶어.")]})
print(state)

{'messages': AIMessage(content='한스 란다 대령과 대화하고 싶으시군요! 확인을 위해 그의 이름과 소속된 영화 유니버스는 다음과 같습니다:\n\n- 이름: 한스 란다\n- 유니버스: 바스터즈: 거친 녀석들 (영화)\n\n이 정보가 맞나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 176, 'total_tokens': 249, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BIB4QkXETd5wz14GNm4tTXg8Jyrli', 'finish_reason': 'stop', 'logprobs': None}, id='run-4eea5f3e-3619-4b66-89f6-b072b5883d62-0', usage_metadata={'input_tokens': 176, 'output_tokens': 73, 'total_tokens': 249, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}
